## Window Function in Spark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [8]:
spark = (
    SparkSession
    .builder \
    .appName("JoinStrategy")\
    .master("local[*]")\
    .getOrCreate()
)

spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/21 01:34:03 WARN Utils: Your hostname, Shrees-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.183.253.103 instead (on interface en0)
25/11/21 01:34:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/21 01:34:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
from pyspark.sql.window import Window

In [14]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

emp_schema = ['id', 'name', 'sal', 'dept', 'gender']

emp_df = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [18]:
emp_df.show()

+---+--------+------+---------+------+
| id|    name|   sal|     dept|gender|
+---+--------+------+---------+------+
|  1|  manish| 50000|       IT|     m|
|  2|  vikash| 60000|    sales|     m|
|  3| raushan| 70000|marketing|     m|
|  4|  mukesh| 80000|       IT|     m|
|  5|   priti| 90000|    sales|     f|
|  6|  nikita| 45000|marketing|     f|
|  7|  ragini| 55000|marketing|     f|
|  8|   rashi|100000|       IT|     f|
|  9|  aditya| 65000|       IT|     m|
| 10|   rahul| 50000|marketing|     m|
| 11|   rakhi| 50000|       IT|     f|
| 12|akhilesh| 90000|    sales|     m|
+---+--------+------+---------+------+



In [22]:
## window function implimentation on 'dept'
window = Window.partitionBy("dept").orderBy("sal")

emp_df.withColumn("row_number", row_number().over(window))\
        .withColumn("Rank",rank().over(window))\
        .withColumn("Dense_Rank", dense_rank().over(window))\
        .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |sal   |dept     |gender|row_number|Rank|Dense_Rank|
+---+--------+------+---------+------+----------+----+----------+
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|11 |rakhi   |50000 |IT       |f     |2         |1   |1         |
|9  |aditya  |65000 |IT       |m     |3         |3   |2         |
|4  |mukesh  |80000 |IT       |m     |4         |4   |3         |
|8  |rashi   |100000|IT       |f     |5         |5   |4         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|10 |rahul   |50000 |marketing|m     |2         |2   |2         |
|7  |ragini  |55000 |marketing|f     |3         |3   |3         |
|3  |raushan |70000 |marketing|m     |4         |4   |4         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|5  |priti   |90000 |sales    |f     |2         |2   |2         |
|12 |akhilesh|90000 |sales    |m     |3         |2   |2         |
+---+-----

In [24]:
## window function implimentation on 'dept' and 'sal'
window = Window.partitionBy("dept","gender").orderBy("sal")

emp_df.withColumn("row_number", row_number().over(window))\
        .withColumn("Rank",rank().over(window))\
        .withColumn("Dense_Rank", dense_rank().over(window))\
        .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |sal   |dept     |gender|row_number|Rank|Dense_Rank|
+---+--------+------+---------+------+----------+----+----------+
|11 |rakhi   |50000 |IT       |f     |1         |1   |1         |
|8  |rashi   |100000|IT       |f     |2         |2   |2         |
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|9  |aditya  |65000 |IT       |m     |2         |2   |2         |
|4  |mukesh  |80000 |IT       |m     |3         |3   |3         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|7  |ragini  |55000 |marketing|f     |2         |2   |2         |
|10 |rahul   |50000 |marketing|m     |1         |1   |1         |
|3  |raushan |70000 |marketing|m     |2         |2   |2         |
|5  |priti   |90000 |sales    |f     |1         |1   |1         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|12 |akhilesh|90000 |sales    |m     |2         |2   |2         |
+---+-----

In [32]:
window = Window.partitionBy("dept","gender").orderBy("sal")

emp_df.withColumn("row_number", row_number().over(window))\
        .withColumn("Rank",rank().over(window))\
        .withColumn("Dense_Rank", dense_rank().over(window))\
        .filter(col("Dense_Rank")<=2)\
        .show(truncate=False)

+---+--------+------+---------+------+----------+----+----------+
|id |name    |sal   |dept     |gender|row_number|Rank|Dense_Rank|
+---+--------+------+---------+------+----------+----+----------+
|11 |rakhi   |50000 |IT       |f     |1         |1   |1         |
|8  |rashi   |100000|IT       |f     |2         |2   |2         |
|1  |manish  |50000 |IT       |m     |1         |1   |1         |
|9  |aditya  |65000 |IT       |m     |2         |2   |2         |
|6  |nikita  |45000 |marketing|f     |1         |1   |1         |
|7  |ragini  |55000 |marketing|f     |2         |2   |2         |
|10 |rahul   |50000 |marketing|m     |1         |1   |1         |
|3  |raushan |70000 |marketing|m     |2         |2   |2         |
|5  |priti   |90000 |sales    |f     |1         |1   |1         |
|2  |vikash  |60000 |sales    |m     |1         |1   |1         |
|12 |akhilesh|90000 |sales    |m     |2         |2   |2         |
+---+--------+------+---------+------+----------+----+----------+

